In [193]:
import pandas as pd
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 30)


import numpy as np

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

# Compiling Dataset

# Data Formatting and Feature Engineering
Using PandaSql, we join relevant tables and variables, and save them as CSV files to be used later.

In [2]:
sellers_df = pd.read_csv('data/source_data/olist_sellers_dataset.csv')
products_df = pd.read_csv('data/source_data/olist_products_dataset.csv')
orders_df = pd.read_csv('data/source_data/olist_orders_dataset.csv')
reviews_df = pd.read_csv('data/source_data/olist_order_reviews_dataset.csv')
payments_df = pd.read_csv('data/source_data/olist_order_payments_dataset.csv')
items_df = pd.read_csv('data/source_data/olist_order_items_dataset.csv')
geo_df = pd.read_csv('data/source_data/olist_geolocation_dataset.csv')
customers_df = pd.read_csv('data/source_data/olist_customers_dataset.csv')

In [3]:
dfs = [sellers_df, products_df, orders_df, reviews_df, payments_df, items_df, geo_df, customers_df]

for df in dfs:
    print(df.columns)

Index(['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'], dtype='object')
Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')
Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')
Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')
Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')
Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value']

# Products

Joined products, sellers, orders, items

feature engineering on dates

averages 

In [295]:
#Products

q = """SELECT p.product_id, p.product_category_name, p.product_name_lenght, p.product_description_lenght, p.product_photos_qty, p.product_weight_g, p.product_length_cm, p.product_height_cm, p.product_width_cm,
               o.order_id, o.customer_id, o.order_purchase_timestamp, o.order_approved_at, o.order_delivered_carrier_date, o.order_delivered_customer_date, o.order_estimated_delivery_date,
               i.order_item_id, i.shipping_limit_date, i.price, i.freight_value,
               s.seller_id, s.seller_zip_code_prefix, s.seller_city, s.seller_state,
               r.review_score
        FROM products_df p
        INNER JOIN items_df i
        ON i.product_id = p.product_id
        INNER JOIN sellers_df s
        ON s.seller_id = i.seller_id
        INNER JOIN orders_df o
        ON o.order_id = i.order_id
        INNER JOIN reviews_df r
        ON o.order_id = r.order_id
        LIMIT 500
        ;"""

df = pysqldf(q)

df.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,shipping_limit_date,price,freight_value,seller_id,seller_zip_code_prefix,seller_city,seller_state,review_score
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,e17e4f88e31525f7deef66779844ddce,f8a3e963a310aa58b60a5b1fed5bceb5,2018-04-24 16:16:53,2018-04-24 19:04:19,2018-04-25 17:01:00,2018-04-27 16:42:17,2018-05-09 00:00:00,1,2018-04-30 17:33:54,10.91,7.39,5670f4db5b62c43d542e1b2d56b0cf7c,3694,sao paulo,SP,5
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,5236307716393b7114b53ee991f36956,03fc97548af8f58fefc768d12b546c9c,2018-01-31 18:55:36,2018-01-31 19:11:15,2018-02-05 22:44:47,2018-02-16 13:27:08,2018-02-22 00:00:00,1,2018-02-06 19:11:15,248.00,17.99,b561927807645834b59ef0d16ba55a24,2955,sao paulo,SP,5
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,01f66e58769f84129811d43eefd187fb,e41819d1c95c12c9ce495b630eab8aee,2018-07-09 21:18:01,2018-07-09 21:30:20,2018-07-10 11:49:00,2018-07-11 23:24:16,2018-07-18 00:00:00,1,2018-07-11 21:30:20,79.80,7.82,7b07b3c7487f0ea825fc6df75abd658b,2016,sao paulo,SP,5
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,143d00a4f2dde4e0364ee1821577adb3,322162b5ca010c2b059cb5224dd818b1,2018-08-03 08:55:50,2018-08-03 09:31:42,2018-08-03 11:48:00,2018-08-06 15:14:32,2018-08-08 00:00:00,1,2018-08-07 09:10:13,112.30,9.54,c510bc1718f0f2961eaa42a23330681a,13330,indaiatuba,SP,1
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,86cafb8794cb99a9b1b77fc8e48fbbbb,c11c31965ff02cc1d7132df8edfcbc22,2018-04-11 01:06:37,2018-04-11 01:30:23,2018-04-12 22:56:27,2018-04-13 22:41:44,2018-04-26 00:00:00,1,2018-04-17 01:30:23,37.90,8.29,0be8ff43f22e456b4e0371b2245e4d01,4461,sao paulo,SP,5


### Delivery

Instead of using physical dates, we extracted the relevent information and used it to **

In [296]:
df.columns

Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'order_id', 'customer_id', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'shipping_limit_date', 'price', 'freight_value',
       'seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state',
       'review_score'],
      dtype='object')

In [297]:
cols = ['order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date','shipping_limit_date']

for col in cols:
    df[col] = pd.to_datetime(df[col], infer_datetime_format=True, errors='coerce')

In [298]:
approval_time = []

for x, time in enumerate(df['order_approved_at']):
    approval_time.append(time - df['order_purchase_timestamp'][x])

In [299]:
total_order_time = []

for x, time in enumerate(df['order_delivered_customer_date']):
    total_order_time.append(time - df['order_purchase_timestamp'][x])

In [300]:
order_estimated_delivery = []

for x, time in enumerate(df['order_purchase_timestamp']):
    order_estimated_delivery.append(time - df['order_estimated_delivery_date'][x])

In [301]:
order_estimation_difference = []

for x, time in enumerate(df['order_delivered_customer_date']):
    order_estimation_difference.append(time - df['order_estimated_delivery_date'][x])

In [302]:
order_shipping_limit = []

for x, time in enumerate(df['shipping_limit_date']):
    order_shipping_limit.append(time - df['order_purchase_timestamp'][x])

In [303]:
df['order_approval_time'] = approval_time
df['total_order_time'] = total_order_time
df['order_estimated_delivery'] = order_estimated_delivery
df['order_estimation_difference'] = order_estimation_difference
df['order_shipping_limit'] = order_shipping_limit

In [304]:
df.drop(columns=cols, inplace=True)

### Delivery Averages per Product

In [371]:
delivery_df = df[['product_id', 'product_category_name', 'order_approval_time', 
                  'total_order_time', 'order_estimated_delivery',
                  'order_estimation_difference', 'order_shipping_limit','customer_id']]

delivery_df

,product_id,product_category_name,order_approval_time,total_order_time,order_estimated_delivery,order_estimation_difference,order_shipping_limit,customer_id
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,0 days 02:47:26,3 days 00:25:24,-15 days +16:16:53,-12 days +16:42:17,6 days 01:17:01,f8a3e963a310aa58b60a5b1fed5bceb5
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,0 days 00:15:39,15 days 18:31:32,-22 days +18:55:36,-6 days +13:27:08,6 days 00:15:39,03fc97548af8f58fefc768d12b546c9c
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,0 days 00:12:19,2 days 02:06:15,-9 days +21:18:01,-7 days +23:24:16,2 days 00:12:19,e41819d1c95c12c9ce495b630eab8aee
3,cef67bcfe19066a932b7673e239eb23d,bebes,0 days 00:35:52,3 days 06:18:42,-5 days +08:55:50,-2 days +15:14:32,4 days 00:14:23,322162b5ca010c2b059cb5224dd818b1
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,0 days 00:23:46,2 days 21:35:07,-15 days +01:06:37,-13 days +22:41:44,6 days 00:23:46,c11c31965ff02cc1d7132df8edfcbc22
...,...,...,...,...,...,...,...,...
495,a5bc1334f1762ce0a844f3f694dc08e1,cama_mesa_banho,1 days 15:18:41,NaT,-23 days +12:10:05,NaT,7 days 15:18:41,9eb0d8ae0777405811966250213e28a2
496,701ff123e9085dcc9c2f833efded6374,cool_stuff,0 days 00:11:44,10 days 20:27:32,-8 days +21:53:39,3 days 18:21:11,7 days 00:11:44,97a0e2c05d5882609289a1816e234ba0
497,701ff123e9085dcc9c2f833efded6374,cool_stuff,1 days 00:01:04,12 days 23:29:14,-26 days +20:09:23,-13 days +19:38:37,7 days 00:01:04,1117174c546ffd2ad76d566a3ea131f2
498,701ff123e9085dcc9c2f833efded6374,cool_stuff,0 days 00:12:14,16 days 11:10:15,-12 days +14:33:08,5 days 01:43:23,6 days 00:12:14,bd574aa2df7e753d954064917212f352


In [348]:
# delivery_df = delivery_df.groupby(['product_category_name','product_id']).agg(lambda x: x.mean())
# delivery_df

In [307]:
# approvals = df['order_approval_time']

# result = map(delivery_df.groupby(['product_id']).agg(lambda x: x.mean()), approvals)

# list(result)

In [398]:
delivery_averages = delivery_df.groupby('product_id').mean(numeric_only=False)
delivery_averages

,order_approval_time,order_estimated_delivery,order_estimation_difference,order_shipping_limit,total_order_time
product_id,,,,,
003938452c98ff9ab28e9e7b4bfe97ab,2 days 07:27:11,-12 days +21:28:12,-6 days +17:44:46,5 days 07:27:11,5 days 20:16:34
0075c14603d80515e20908c156ceb47b,0 days 00:19:52,-24 days +02:15:42,-16 days +15:12:48,7 days 00:15:18,8 days 12:57:06
03b63c5fc16691530586ae020c345514,0 days 14:12:00.500000,-38 days +03:15:55.500000,-21 days +19:48:56.500000,7 days 12:45:17,17 days 16:33:01
051b9ff13dd55c0a6655a15ff296f80d,0 days 00:12:34,-12 days +13:57:47,-5 days +20:43:33,6 days 00:12:34,7 days 06:45:46
07f01b6fcacc1b187a71e5074199db2d,0 days 17:14:26,-56 days +00:14:19,-43 days +16:04:48,6 days 17:14:26,13 days 15:50:29
...,...,...,...,...,...
f908df99196805b5bc4ada9ef510b11b,0 days 10:14:59.666666,-24 days +15:03:42.666666,-16 days +05:05:15,5 days 02:14:49.666666,7 days 14:01:32.333333
f9471562478eba8761bc985b968a0092,1 days 23:39:09.250000,-31 days +13:03:08.875000,-21 days +22:11:11.375000,7 days 17:29:13.125000,10 days 09:08:02.500000
f9ecf420c38b90a8af8e7f70fd789264,0 days 00:14:49,-32 days +14:29:32,-17 days +17:04:28,4 days 00:14:49,15 days 02:34:56


In [401]:
lst = []

for x, z in enumerate(delivery_df['product_id']):
    for y in delivery_averages.index:
        if z == y:
            lst.append({'product_id': z,
                        'order_approval_time_av':delivery_averages.loc[z]['order_approval_time'],
                         'order_approval_time':delivery_df.iloc[x]['order_approval_time']
                       })

In [402]:
lst

[{'product_id': '1e9e8ef04dbcff4541ed26657ea517e5',
  'order_approval_time_av': Timedelta('0 days 02:47:26'),
  'order_approval_time': Timedelta('0 days 02:47:26')},
 {'product_id': '3aa071139cb16b67ca9e5dea641aaa2f',
  'order_approval_time_av': Timedelta('0 days 00:15:39'),
  'order_approval_time': Timedelta('0 days 00:15:39')},
 {'product_id': '96bd76ec8810374ed1b65e291975717f',
  'order_approval_time_av': Timedelta('0 days 00:12:19'),
  'order_approval_time': Timedelta('0 days 00:12:19')},
 {'product_id': 'cef67bcfe19066a932b7673e239eb23d',
  'order_approval_time_av': Timedelta('0 days 00:35:52'),
  'order_approval_time': Timedelta('0 days 00:35:52')},
 {'product_id': '9dc1a7de274444849c219cff195d0b71',
  'order_approval_time_av': Timedelta('0 days 00:23:46'),
  'order_approval_time': Timedelta('0 days 00:23:46')},
 {'product_id': '41d3672d4792049fa1779bb35283ed13',
  'order_approval_time_av': Timedelta('1 days 00:06:48'),
  'order_approval_time': Timedelta('1 days 00:06:48')},
 {'p

In [375]:
# df.join(delivery_averages, on=delivery_averages.index, rsuffix='_average')

In [317]:
products = df

# Customers

Will be used for Clustering for better insight

In [38]:
#Customers

q = """SELECT c.customer_id, c.customer_zip_code_prefix, c.customer_city, 
                c.customer_state,
                p.payment_value, i.price, i.freight_value,
                p.payment_type, p.payment_installments, i.order_item_id, 
                pd.product_category_name,
                s.seller_city, s.seller_state
        FROM customers_df c
        INNER JOIN orders_df o
        ON o.customer_id = c.customer_id
        INNER JOIN payments_df p
        ON o.order_id = p.order_id
        INNER JOIN items_df i
        ON o.order_id = i.order_id
        INNER JOIN products_df pd
        ON i.product_id = pd.product_id
        INNER JOIN sellers_df s
        ON i.seller_id = s.seller_id
        LIMIT 5
        ;"""

data = pysqldf(q)

data['items_in_order'] = data['order_item_id']
data.drop(columns=['order_item_id'], inplace=True)

data.head()

customers = data

In [37]:
data.columns

Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'seller_id',
       'seller_zip_code_prefix', 'seller_city', 'seller_state', 'order_id',
       'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

# Reviews

Potential for Sentiment Analysis

Length of review, creation date

In [195]:
#Reviews

q = """SELECT r.review_id, r.order_id, r.review_score, r.review_comment_title, r.review_comment_message
        FROM reviews_df r
        WHERE r.review_comment_message NOT NULL
        LIMIT 5
        ;"""

data = pysqldf(q)

data.head()


,review_id,order_id,review_score,review_comment_title,review_comment_message
0,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.
1,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...
2,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...
3,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,None,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes..."


In [229]:
title_lengths = []

for x, length in enumerate(data['review_comment_title']):
    if length == None:
        title_lengths.append('None')
    else:
        title_lengths.append(len(length))
        
        
review_lengths = []

for x in data['review_comment_message']:
    review_lengths.append(len(x))
    
data['title_lengths'] = title_lengths
data['review_lengths'] = review_lengths

In [257]:
reviews = data

# Full Dataset

For model building

In [194]:
reviews_df.columns

Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

In [42]:
#Full Dataset

q = """SELECT p.product_id, p.product_category_name, p.product_name_lenght, p.product_description_lenght, p.product_photos_qty, p.product_weight_g, p.product_length_cm, p.product_height_cm, p.product_width_cm,
               o.order_id, o.customer_id, o.order_purchase_timestamp, o.order_approved_at, o.order_delivered_carrier_date, o.order_delivered_customer_date, o.order_estimated_delivery_date,
               i.order_item_id, i.shipping_limit_date, i.price, i.freight_value,
               s.seller_id, s.seller_zip_code_prefix, s.seller_city, s.seller_state,
               r.review_score, 
               
        FROM products_df p
        INNER JOIN items_df i
        ON i.product_id = p.product_id
        INNER JOIN sellers_df s
        ON s.seller_id = i.seller_id
        INNER JOIN orders_df o
        ON o.order_id = i.order_id
        INNER JOIN reviews_df r
        ON o.order_id = r.order_id
        LIMIT 500
        ;"""
data = pysqldf(q)

data.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,...,payment_type,payment_installments,payment_value,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,06b8999e2fba1a1fbc88172c00ba8bc7,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,...,credit_card,2,146.87,00e7ee1b050b8499577073aeb2a297a1,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,18955e83d337fd6b2def6b18a428ac77,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,...,credit_card,8,335.48,29150127e6685892b6eab3eec79f59c7,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,4e7b3e00288586ebd08712fdd0374a03,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,...,credit_card,7,157.73,b2059ed67ce144a36e2aa97d2c9e9ad2,1,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,b2b6027bc5c5109e529d4dc6358b12c3,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,...,credit_card,1,173.30,951670f92359f4fe4a63112aa7306eba,1,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,4f2d8ab171c80ec8364f7c12e35b23ad,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,...,credit_card,8,252.25,6b7d50bd145f6fc7f33cebabd7e49d0f,1,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25


In [167]:
data.columns

Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'order_id', 'customer_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date'],
      dtype='object')

In [2]:
df1 = pd.read_csv('data/products_final.csv')

In [148]:
df = pd.read_csv('data/products_full.csv')

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (9,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#Order Information Dataset

df['payment_installments'] = df['payment_installments'] .astype('int')

In [173]:
df.columns

cols = ['product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm']

for col in cols:
    df[col] = df[col].astype('int')

In [35]:
orders_df.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

# Order Payment


Payment Methods were encoded with the following labels:
1 = Boleto
2 = Credit Card
3 = Debit Card
4 = Voucher

Payment sequence was removed, as we ony have one payment method per order.

In [153]:
df['payment_type'].value_counts()

credit_card    86012
boleto         22692
voucher         6211
debit_card      1666
Name: payment_type, dtype: int64

In [162]:
pays = df['payment_type']
unexpected = []

for x, pay in enumerate(pays):
    if '2017' not in pay:
        continue
    else:
        unexpected.append(df.index[x])

In [167]:
print(len(unexpected),' unexpected entries dropped')
df.drop(index=unexpected, inplace=True)

0  unexpected entries dropped


In [155]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

In [158]:
df['payment_type'] = df['payment_type'].astype('category')
df['payment_types'] = label.fit_transform(df['payment_type'])
df['payment_method'] = df['payment_types'] + 1

In [170]:
df[['payment_sequential', 'payment_method','payment_installments', 'payment_value']]

,payment_sequential,payment_method,payment_installments,payment_value
0,1,2,1,18.12
1,2,4,1,18.59
2,3,4,1,2.00
3,1,1,1,141.46
4,1,2,3,179.12
...,...,...,...,...
116579,1,2,3,195.00
116580,1,2,5,271.01
116581,1,2,4,441.16
116582,1,2,4,441.16


# Customer Location